In [ ]:
#importing_packages
import pandas as pd
import numpy as np
import pickle
import math
import warnings
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings("ignore")

In [ ]:
#data_import
calendar = pd.read_csv("calendar.csv")
sales_data = pd.read_csv("sales_train_evaluation.csv")
sell_prices = pd.read_csv("sell_prices.csv")

infile = open('encoding','rb')
encoding_dict = pickle.load(infile)
infile.close()

infile = open('best_models','rb')
best_model = pickle.load(infile)
infile.close()

In [ ]:
#sample_input
product = "HOBBIES_1_069"
store = "CA_1"

In [ ]:
#function to return predictions

def final_1(product,store):
  """takes product and store ID as input and returns predictions"""
  x = product + str("_") + store + str("_evaluation")
  x_sales_data = sales_data[sales_data.id==x].reset_index(drop=True)
  for i in range(1942,1970):
    x_sales_data["d_"+str(i)]=np.nan
  x_data = pd.melt(x_sales_data,id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],var_name='day_no')
    
  x_sell_prices = sell_prices[(sell_prices.store_id==store) & (sell_prices.item_id==product)]
  calendar_data = calendar.rename(columns={'d': 'day_no'})
  calendar_data['event_name_1'].fillna("no_event",inplace=True)
  calendar_data['event_type_1'].fillna("no_event",inplace=True)
  calendar_data['event_name_2'].fillna("no_event",inplace=True)
  calendar_data['event_type_2'].fillna("no_event",inplace=True)

  ds=[]
  for i in calendar_data.date:
    m=i.split("-")[2]
    ds.append(m)
  calendar_data.date=ds

  x_data = pd.merge(x_data,calendar_data,how='left',on='day_no')
  x_data.day_no = x_data['day_no'].apply(lambda x: x.split('_')[1]).astype(np.int16)
  x_data = pd.merge(x_data,x_sell_prices,on=['store_id', 'item_id', 'wm_yr_wk'],how='left')
  temp = x_data[['id','day_no','sell_price']]
  temp = temp.pivot(index='id',columns='day_no',values='sell_price')
  temp.interpolate(limit_direction='backward',inplace=True,axis=1)
  temp = pd.melt(temp,ignore_index=False,value_name='sell_price')
  temp = temp.reset_index(level=0)
  x_data['imputed']=x_data['sell_price'].isna()
  x_data = x_data.drop(['sell_price'],axis=1)
  x_data = pd.merge(x_data,temp,on=['id','day_no'])

  state = store.split("_")[0]
  temp = ["CA","TX","WI"]
  temp = [i for i in temp if i!=state]
  x_data.rename(columns = {'snap_'+str(state):'SNAP'}, inplace = True)
  x_data = x_data.drop(["snap_"+str(temp[0]),"snap_"+str(temp[1])],axis=1)

  x_data=x_data.drop("weekday",axis=1)
  x_data["is_Weekend"]=(x_data.wday<2).astype("bool")
  x_data["SNAP"] = x_data["SNAP"].astype('bool')

  shiftt = 28
  for i in range(6):
    x_data["lag_"+str(shiftt+(7*i))] = x_data.groupby("id")["value"].shift(shiftt+(7*i)).fillna(0).astype(np.int16)

  for i in range(1,7):
    x_data["rolling_"+str(7*i)] = x_data.groupby("id")["lag_28"].transform(lambda s: s.rolling(i).mean())

  x_data = x_data[(x_data.day_no>=1069)]

  temp = x_data[["id","sell_price","value"]][(x_data.day_no<1914)].copy()
  temp['revenue']=(temp.value*temp.sell_price).values
  temp=temp.groupby(["id"])["revenue"].sum()
  temp=pd.DataFrame(temp)
  temp.reset_index(inplace=True)
  x_data=pd.merge(x_data,temp,on='id')

  temp=x_data[(x_data.day_no<1914)][["id","wday","state_id","SNAP","value"]].copy()
  temp.rename(columns = {'value':'trend_value'}, inplace = True)
  temp=temp.groupby(["id","wday","SNAP","state_id"])["trend_value"].mean()
  temp=pd.DataFrame(temp)
  temp.reset_index(inplace=True)
  x_data=pd.merge(x_data,temp,on=["id","wday","SNAP","state_id"])

  x_data = x_data[(x_data.day_no>=1942)]

  cat_cols = ['id','item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'date', 'wday', 'month',
            'year', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'SNAP']
  for col in cat_cols:
      x_data[col] = x_data[col].astype('category')
      lenc = encoding_dict[col]
      x_data[col] = lenc.transform(x_data[col].astype(str)).astype(np.int16)

  icols = x_data.select_dtypes('integer').columns
  fcols = x_data.select_dtypes('float').columns
  x_data[icols] = x_data[icols].apply(pd.to_numeric, downcast='integer')
  x_data[fcols] = x_data[fcols].apply(pd.to_numeric, downcast='float')

  x_data = x_data.drop(['value'],axis=1)

  states = {"CA":0 , "TX":1 , "WI":2}
  lgb_reg_best = best_model[states[state]]

  custom_predictions = pd.DataFrame()
  custom_predictions["id"] = x_sales_data["id"].copy()
  for a,l in enumerate(range(1942,1970)):
        custom_predictions['F'+str(a+1)] = lgb_reg_best.predict(x_data[x_data['day_no'] == l])

  return custom_predictions

In [ ]:
j=final_1(product,store)
j

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_069_CA_1_evaluation,0.32188,0.313074,0.356796,0.234716,0.263155,0.382095,0.345813,0.247414,0.234344,...,0.358209,0.454557,0.380881,0.376198,0.396506,0.423041,0.313618,0.397833,0.530039,0.399325


In [ ]:
#function to return rmsse value

def final_2(product,store):
  """takes product and store ID as input and returns rmsse score"""
  x = product + str("_") + store + str("_evaluation")
  x_sales_data = sales_data[sales_data.id==x].reset_index(drop=True)
  for i in range(1942,1970):
    x_sales_data["d_"+str(i)]=np.nan
  x_data = pd.melt(x_sales_data,id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],var_name='day_no')
    
  x_sell_prices = sell_prices[(sell_prices.store_id==store) & (sell_prices.item_id==product)]
  calendar_data = calendar.rename(columns={'d': 'day_no'})
  calendar_data['event_name_1'].fillna("no_event",inplace=True)
  calendar_data['event_type_1'].fillna("no_event",inplace=True)
  calendar_data['event_name_2'].fillna("no_event",inplace=True)
  calendar_data['event_type_2'].fillna("no_event",inplace=True)

  ds=[]
  for i in calendar_data.date:
    m=i.split("-")[2]
    ds.append(m)
  calendar_data.date=ds

  x_data = pd.merge(x_data,calendar_data,how='left',on='day_no')
  x_data.day_no = x_data['day_no'].apply(lambda x: x.split('_')[1]).astype(np.int16)
  x_data = pd.merge(x_data,x_sell_prices,on=['store_id', 'item_id', 'wm_yr_wk'],how='left')
  temp = x_data[['id','day_no','sell_price']]
  temp = temp.pivot(index='id',columns='day_no',values='sell_price')
  temp.interpolate(limit_direction='backward',inplace=True,axis=1)
  temp = pd.melt(temp,ignore_index=False,value_name='sell_price')
  temp = temp.reset_index(level=0)
  x_data['imputed']=x_data['sell_price'].isna()
  x_data = x_data.drop(['sell_price'],axis=1)
  x_data = pd.merge(x_data,temp,on=['id','day_no'])

  state = store.split("_")[0]
  temp = ["CA","TX","WI"]
  temp = [i for i in temp if i!=state]
  x_data.rename(columns = {'snap_'+str(state):'SNAP'}, inplace = True)
  x_data = x_data.drop(["snap_"+str(temp[0]),"snap_"+str(temp[1])],axis=1)

  x_data=x_data.drop("weekday",axis=1)
  x_data["is_Weekend"]=(x_data.wday<2).astype("bool")
  x_data["SNAP"] = x_data["SNAP"].astype('bool')

  shiftt = 28
  for i in range(6):
    x_data["lag_"+str(shiftt+(7*i))] = x_data.groupby("id")["value"].shift(shiftt+(7*i)).fillna(0).astype(np.int16)

  for i in range(1,7):
    x_data["rolling_"+str(7*i)] = x_data.groupby("id")["lag_28"].transform(lambda s: s.rolling(i).mean())

  x_data = x_data[(x_data.day_no>=1069)]

  temp = x_data[["id","sell_price","value"]][(x_data.day_no<1914)].copy()
  temp['revenue']=(temp.value*temp.sell_price).values
  temp=temp.groupby(["id"])["revenue"].sum()
  temp=pd.DataFrame(temp)
  temp.reset_index(inplace=True)
  x_data=pd.merge(x_data,temp,on='id')

  temp=x_data[(x_data.day_no<1914)][["id","wday","state_id","SNAP","value"]].copy()
  temp.rename(columns = {'value':'trend_value'}, inplace = True)
  temp=temp.groupby(["id","wday","SNAP","state_id"])["trend_value"].mean()
  temp=pd.DataFrame(temp)
  temp.reset_index(inplace=True)
  x_data=pd.merge(x_data,temp,on=["id","wday","SNAP","state_id"])

  x_data = x_data[(x_data.day_no>=1914)]

  cat_cols = ['id','item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'date', 'wday', 'month',
            'year', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'SNAP']
  for col in cat_cols:
      x_data[col] = x_data[col].astype('category')
      lenc = encoding_dict[col]
      x_data[col] = lenc.transform(x_data[col].astype(str)).astype(np.int16)

  icols = x_data.select_dtypes('integer').columns
  fcols = x_data.select_dtypes('float').columns
  x_data[icols] = x_data[icols].apply(pd.to_numeric, downcast='integer')
  x_data[fcols] = x_data[fcols].apply(pd.to_numeric, downcast='float')

  x_data = x_data.drop(['value'],axis=1)

  states = {"CA":0 , "TX":1 , "WI":2}
  lgb_reg_best = best_model[states[state]]

  custom_predictions = pd.DataFrame()
  for l in range(1914,1942):
        custom_predictions['d_'+str(l)] = lgb_reg_best.predict(x_data[x_data['day_no'] == l])

  cv_data = x_sales_data.loc[:,'d_'+str(1914):"d_"+str(1941)]
  predi = custom_predictions.loc[:,'d_'+str(1914):"d_"+str(1941)]
  rmse=((cv_data-predi)**2).mean(axis=1)

  data = x_sales_data.loc[:,'d_'+str(1):"d_"+str(1913)]
  row = data.loc[0].values
  row = row[np.argmax(row!=0):]
  den = ((row[1:]-row[:-1])**2).mean()

  rmsse = np.sqrt(rmse/den)
  return rmsse

In [ ]:
s=final_2(product,store)
s

0    0.493593
dtype: float64